In [5]:
import pandas as pd
import numpy as np
import re
from itertools import combinations

# Muat dataset resep dengan penanganan error
recipes_df = pd.read_csv('dataset_finale.csv')

# Validasi kolom dataset
required_columns = ['Title', 'Kalori', 'Ingredients', 'Steps', 'Loves']
missing_columns = [col for col in required_columns if col not in recipes_df.columns]
if missing_columns:
    raise KeyError(f"Kolom berikut tidak ditemukan di dataset: {missing_columns}")

# Pastikan kolom 'Ingredients' berisi string
recipes_df['Ingredients'] = recipes_df['Ingredients'].astype(str)

# Pastikan kolom 'Loves' berisi numerik, isi NaN dengan 0
recipes_df['Loves'] = pd.to_numeric(recipes_df['Loves'], errors='coerce').fillna(0)

# Fungsi untuk memeriksa apakah resep mengandung semua bahan yang diberikan
def contains_all_ingredients(recipe_ingredients, required_ingredients):
    recipe_items = [item.lower() for item in recipe_ingredients.split(', ')]
    return all(any(req_ing in item for item in recipe_items) for req_ing in required_ingredients)

# Fungsi rekomendasi resep dengan pencarian bertahap
def recommend_recipes(ingredients, recipes_df, max_ingredients=5):
    if not ingredients or not all(isinstance(ing, str) and ing.strip() for ing in ingredients):
        return "Input bahan tidak valid atau kosong."
    
    ingredients = [ing.strip().lower() for ing in ingredients]
    result = []
    
    # Batasi maksimal 5 bahan
    ingredients = ingredients[:max_ingredients]
    num_ingredients = len(ingredients)
    
    # Mulai dari jumlah bahan maksimal hingga minimal 1 bahan
    for n in range(num_ingredients, 0, -1):
        all_combinations = list(combinations(ingredients, n))
        matching_recipes = pd.DataFrame()
        
        for comb in all_combinations:
            comb_list = list(comb)
            comb_matches = recipes_df[recipes_df['Ingredients'].apply(lambda x: contains_all_ingredients(x, comb_list))]
            matching_recipes = pd.concat([matching_recipes, comb_matches]).drop_duplicates()
        
        if not matching_recipes.empty:
            # Urutkan berdasarkan Loves
            top_3 = matching_recipes.sort_values(by='Loves', ascending=False).head(3)
            result.extend(top_3[['Title', 'Kalori', 'Ingredients', 'Steps']].to_dict('records'))
            break  # Berhenti jika sudah menemukan kombinasi yang cocok
    
    if not result:
        result.append("Tidak ada resep yang ditemukan untuk kombinasi bahan tersebut.")
    
    return result

# Fungsi untuk memecah Ingredients dan Steps menjadi daftar
def format_recipe_details(recipe):
    ingredients_list = [item.strip() for item in recipe['Ingredients'].split(', ')]
    steps_text = recipe['Steps']
    # Pisahkan langkah berdasarkan pola angka dan titik (termasuk angka dua digit)
    steps_list = [step.strip() for step in re.split(r'(?=\d+\.\s)', steps_text) if step.strip()]
    # Hapus duplikat langkah
    seen_steps = set()
    unique_steps = []
    for step in steps_list:
        if step not in seen_steps:
            seen_steps.add(step)
            unique_steps.append(step)
    return ingredients_list, unique_steps

# Inputan manual
manual_input = "telur cabai bawang merah garam"
ingredients = manual_input.split()
result = recommend_recipes(ingredients, recipes_df)

if isinstance(result, str):
    print(result)
else:
    print(f"Hasil pencarian untuk bahan {', '.join(ingredients)}:")
    for item in result:
        if isinstance(item, str):
            print(item)
        else:
            print(f"\n{item['Title']}")
            print(f"Kalori: {item['Kalori']}")
            print("**Ingredients**:")
            ingredients_list, steps_list = format_recipe_details(item)
            for ing in ingredients_list:
                print(ing)
            print("**Steps**:")
            for step in steps_list:
                # Tambahkan \n setelah langkah yang dimulai dengan angka dan titik
                if re.match(r'^\d+\.\s', step):
                    print(f"{step}\n")
                else:
                    print(step)
        print()

Hasil pencarian untuk bahan telur, cabai, bawang, merah, garam:

Kalio Talua/Telur & Tempe #pr_recookmasakanawalanT
Kalori: 3250.0
**Ingredients**:
1. 6 butir telur ayam
rebus sampai matang
kupas
sisihkan
2. 1/2 papan tempe
potong sesuai selera
3. Secukupnya santan dari 1 butir kelapa
4. 10 buah cabai rawit merah utuh
5. 1 lembar daun kunyit simpulkan
6. 1 lembar daun salam
7. 2 lembar daun jeruk
8. 1 batang serai
memarkan
9. Secukupnya garam
gula
dan kaldu bubuk jika suka
10. Secukupnya minyak untuk menumis
11. Bumbu yang dihaluskan :
12. 5 buah cabai merah besar
13. 5 siung bawang merah
14. 3 siung bawang putih
15. 1 ruas jari laos/lengkuas
16. 1 ruas jari jahe
17. 1 ruas jari kunyit
**Steps**:
1. Tuang air santan kedalam panci, sisihkan. Tumis bumbu halus dan rempah daun menggunakan minyak sayur sampai harum.(resep asli gak ditumis dulu).

2. Kemudian tuang tumisan bumbu kedalam air santan yang didalam panci. Aduk rata. Nyalakan api. Masak sambil terus diaduk supaya santan tidak pec